# Genetic Algorithm voor Job Scheduling (PI7)
  
Doel: experimenteer met selectie, crossover en mutatie en evalueer combinaties van populatiegrootte, mutatiegraad en aantal iteraties.

Structuur:
- Data laden en verkennen
- Functies: fitness, selectie, crossover, mutatie
- Uitvoering van GA en visualisatie van voortgang
- Parameter sweep en evaluatie van configuraties
- Conclusies en suggesties voor vervolg

In [292]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

## Dataset en preprocessing

- De dataset wordt ingelezen uit `pi-7-voorbeeld-dataset.csv`.
- Als er geen expliciete id-kolom is, wordt een `job_id` toegevoegd op basis van de rijindex.
- Vervolgens sorteer ik op `profit` (desc) — dit verandert alleen de weergegeven volgorde; interne job indices blijven consistent met de oorspronkelijke rijen.

In [293]:
df = pd.read_csv('pi-7-voorbeeld-dataset.csv')  # Importeren van dataset

# bewaar originele id (als CSV geen id-kolom heeft, gebruik rijindex)
if 'id' not in df.columns:
    df['job_id'] = df.index  # of use df.index+1 voor 1-based ids
else:
    df['job_id'] = df['id']

print("\nVoor sorteren:")
print(df)

df = df.sort_values(by=['profit'], ascending=False).reset_index(drop=True)
# let op: reset_index verandert rijindices; job_id blijft de originele id
print("\nNa sorteren:")
print(df)


Voor sorteren:
      id  deadline  profit  job_id
0     95         6     100      95
1     23        10     100      23
2    155         6      99     155
3     57         2      97      57
4    162         8      97     162
..   ...       ...     ...     ...
195   63         8       3      63
196  118         3       2     118
197   55         2       1      55
198  181         3       1     181
199   15        10       1      15

[200 rows x 4 columns]

Na sorteren:
      id  deadline  profit  job_id
0     95         6     100      95
1     23        10     100      23
2    155         6      99     155
3     57         2      97      57
4    162         8      97     162
..   ...       ...     ...     ...
195   63         8       3      63
196  118         3       2     118
197   55         2       1      55
198  181         3       1     181
199   15        10       1      15

[200 rows x 4 columns]


## GA: functies en ontwerpkeuzes

- Representatie: permutaties van job-indexen (volgorde), geschikt voor job sequencing.
- Fitness: berekent totale profit door jobs op de laatste vrije positie vóór hun deadline te plaatsen.
- Parent selection: stochastische selectie op basis van fitness (fallback: willekeurig als alle fitness ≤ 0).
- Crossover: Order Crossover (OX) geschikt voor permutaties.
- Mutatie: swap-mutation (kleine kans om twee genen te wisselen).

In [294]:
def initialise_population(n_pop, n_jobs):
    # Genereer n_pop lijsten, elk een willekeurige volgorde van 0..n_jobs-1
    return [random.sample(range(n_jobs), n_jobs) for i in range(n_pop)]

def fitness(solution, df):
    n = len(df)
    deadlines = df['deadline'].tolist()
    profits = df['profit'].tolist()
    schedule = [None] * n  # hierin komt welke job per slot
    for job in solution:
        deadline = min(deadlines[job], n)  # max n
        # zoek het laatste vrije slot vóór de deadline
        while deadline > 0 and schedule[deadline-1] is not None:
            deadline -= 1
        if deadline > 0:
            schedule[deadline-1] = job
    # tel de winst op van alle geplande jobs
    total_profit = sum(profits[j] for j in schedule if j is not None)
    return total_profit

def evaluate(population, df):
    return [fitness(sol, df) for sol in population]

def parent_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    if total_fitness <= 0:
        # fallback: kies twee willekeurige ouders als gewichten ongeldig zijn
        return random.sample(population, 2)
    selection_probs = [f / total_fitness for f in fitness_scores]
    parents = random.choices(population, weights=selection_probs, k=2)
    return parents

def order_crossover(parents):
    # vervangt single_point_crossover voor permutaties (Order Crossover)
    p1, p2 = parents[0], parents[1]
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    child1 = [-1]*n
    child1[a:b+1] = p1[a:b+1]
    fill = (b+1) % n
    for i in range(n):
        gene = p2[(b+1+i) % n]
        if gene not in child1:
            child1[fill] = gene
            fill = (fill+1) % n
    child2 = [-1]*n
    child2[a:b+1] = p2[a:b+1]
    fill = (b+1) % n
    for i in range(n):
        gene = p1[(b+1+i) % n]
        if gene not in child2:
            child2[fill] = gene
            fill = (fill+1) % n
    return child1, child2

def mutate_swap(individual, rate):
    if random.random() < rate:
        m1, m2 = random.sample(range(len(individual)), 2)
        individual[m1], individual[m2] = individual[m2], individual[m1]
    return individual

def print_schedule(solution, df):
    n = len(df)
    deadlines = df['deadline'].tolist()
    profits = df['profit'].tolist()
    schedule = [None] * n  # elk index = tijdslot

    for job in solution:
        deadline = min(deadlines[job], n)
        while deadline > 0 and schedule[deadline-1] is not None:
            deadline -= 1
        if deadline > 0:
            schedule[deadline-1] = job

    total_profit = 0
    # Print overzicht per slot
    print("\nBeste schedule per tijdslot:")
    for slot, job in enumerate(schedule, start=1):
        if job is not None:
            profit = float(df.loc[job,'profit'])
            deadline = int(df.loc[job,'deadline'])
            job_id = int(df.loc[job,'job_id'])
            total_profit += profit
            print(f"Slot {slot}: job_id={job_id}, profit={profit}, deadline={deadline}")

    print(f"\n>>> Totale profit = {total_profit}")

    return schedule,total_profit


## Experimenteeropzet

- Grid search over:
  - populatiegroottes: [20, 50, 100, 200]
  - mutatiepercentages: [0.01, 0.05, 0.1, 0.3]
  - iteraties: [50, 100, 200, 400]
- Per combinatie meerdere herhalingen (repeats) voor gemiddelde en spreiding.
- Resultaten: mean_score, std_score, best_score en gemiddelde runtijd.

In [ ]:
def run_ga_once(df, n_pop, mutation_rate, n_iters, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    n_jobs = len(df)
    population = initialise_population(n_pop, n_jobs)
    best_score = -float('inf')
    best_ind = None
    for it in range(1, n_iters+1):
        fitness_scores = evaluate(population, df)
        new_pop = []
        while len(new_pop) < n_pop:
            parents = parent_selection(population, fitness_scores)
            c1, c2 = order_crossover(parents)
            new_pop.append(mutate_swap(c1, mutation_rate))
            if len(new_pop) < n_pop:
                new_pop.append(mutate_swap(c2, mutation_rate))
        population = new_pop
        fitness_scores = evaluate(population, df)
        idx_best = max(range(len(population)), key=lambda i: fitness_scores[i])
        if fitness_scores[idx_best] > best_score:
            best_score = fitness_scores[idx_best]
            best_ind = population[idx_best].copy()
    return best_score, best_ind

# parameter grid inclusief iteraties
pop_sizes = [20, 50, 100, 200]
mutation_rates = [0.01, 0.05, 0.1, 0.3]
n_iters_list = [50, 100, 200, 400]  # test verschillende aantallen iteraties
repeats = 2                         # aantal herhalingen per combinatie (verhoog voor betrouwbaarder resultaat)

records = []
best_overall = {'score': -float('inf')}

for n_iters in n_iters_list:
    for p in pop_sizes:
        for m in mutation_rates:
            scores = []
            times = []
            for r in range(repeats):
                t0 = time.time()
                score, ind = run_ga_once(df, n_pop=p, mutation_rate=m, n_iters=n_iters, seed=r)
                t1 = time.time()
                scores.append(score)
                times.append(t1 - t0)
                if score > best_overall['score']:
                    best_overall = {
                        'score': score,
                        'individual': ind,
                        'pop': p,
                        'mut': m,
                        'n_iters': n_iters,
                        'seed': r
                    }
            records.append({
                'pop': p,
                'mut': m,
                'n_iters': n_iters,
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
                'best_score': np.max(scores),
                'mean_time_sec': np.mean(times)
            })

res_df = pd.DataFrame(records)

# duidelijke tekstoutput
print("\nTop 10 configuraties op mean_score:")
print(res_df.sort_values(['mean_score'], ascending=False).head(10).to_string(index=False))

best_row = res_df.loc[res_df['mean_score'].idxmax()]
print("\nBeste configuratie (op basis van mean_score):")
print(best_row.to_string())

print("\nBeste individuele run gevonden over alle tests:")
print(f"score={best_overall['score']}, pop={best_overall['pop']}, mut={best_overall['mut']}, n_iters={best_overall['n_iters']}, seed={best_overall['seed']}")

# toon schedule van beste individuele run
print("\nSchedule voor beste individuele run:")
if best_overall.get('individual') is not None:
    print_schedule(best_overall['individual'], df)

Uit de test kwam vaak 
populatie: 100
Iterations: 400 
Mutation_rate: 0.05

Als beste GA waarden met hoogste profit

In [ ]:
# GA parameters
N_POP = 100
N_JOBS = len(df)
N_ITERS = 400
MUTATION_RATE = 0.05

# Maak populatie aan (lijst van n_pop willekeurige volgorde van n_jobs)
population = initialise_population(N_POP, N_JOBS)

fitness_scores = evaluate(population, df)

best_overall = None
best_score = -float('inf')  # hoger is beter (jouw fitness retourneert total_profit)
best_so_far = -float('inf')
history_best = []

for it in range(1, N_ITERS+1):
    fitness_scores = evaluate(population, df)
    best_iter = max(fitness_scores)
    if best_iter > best_so_far:
        best_so_far = best_iter

    history_best.append(best_so_far)
    # vul nieuwe populatie volledig met nakomelingen (geen elitisme)
    new_pop = []
    while len(new_pop) < N_POP:
        parents = parent_selection(population, fitness_scores)
        c1, c2 = order_crossover(parents)
        # gebruik bestaande mutate_swap functie (geeft permutatie terug)
        new_pop.append(mutate_swap(c1, MUTATION_RATE))
        if len(new_pop) < N_POP:
            new_pop.append(mutate_swap(c2, MUTATION_RATE))

    population = new_pop
    fitness_scores = evaluate(population, df)

    # update beste oplossing
    idx_best = max(range(len(population)), key=lambda i: fitness_scores[i])
    if fitness_scores[idx_best] > best_score:
        best_score = fitness_scores[idx_best]
        best_overall = population[idx_best].copy()
        
    if it == 1 or it % 20 == 0:
        print(f"Iter {it}/{N_ITERS} best profit: {best_score}")

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history_best, label="Beste profit")

plt.xlabel("Iteratie")
plt.ylabel("Profit")
plt.title("GA progressie: profit")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
best_schedule_slots = print_schedule(best_overall, df)


## Conclusies en vervolgstappen

- Samenvatting van observaties (vul in na experimenten).
- Mogelijke verbeteringen:
  - Elitisme toevoegen (behouden beste individuen).
  - Extra crossover- en mutatie-varianten (PMX, inversion, scramble).
  - Alternatieve selectie (tournament selection).
  - Parallelisatie of vectorisatie van fitness-evaluatie voor grotere datasets.
- Referenties: voeg korte literatuurtips toe (bijv. Goldberg 1989; Michalewicz 1996).